# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

*Note: If you are using a Windows machine, you are encouraged to use [7zip](http://www.7-zip.org/) to extract the folder.*

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [1]:
import numpy as np
from glob import glob
from datetime import datetime, timedelta

# load filenames for human and dog images
human_files = np.array(glob("data/lfw/*/*"))
dog_files = np.array(glob("data/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.


<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# extract pre-trained face detector
# face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[0])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

In [ ]:
def show_face(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    for (x,y,w,h) in faces:
        # add bounding box to color image
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

    # convert BGR image to RGB for plotting
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # display the image, along with bounding box
    plt.imshow(cv_rgb)
    plt.show()

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 
(You can print out your results and/or write your percentages in this cell)

In [ ]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

#-#-# Do NOT modify the code above this line. #-#-#

## TODO: Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.
face_det_res = [face_detector(human_files_short[i]) for i in tqdm(range(len(human_files_short)))]
print("Face detector:", len(np.nonzero(face_det_res)[0]), '%')

face_det_dogs = [face_detector(dog_files_short[i]) for i in tqdm(range(len(dog_files_short)))]
print("Face detector on dogs:", len(np.nonzero(face_det_dogs)[0]), '%')

In [ ]:
np.nonzero(face_det_dogs)[0]

In [ ]:
# show all face positive cases on dog images
for idx in np.nonzero(face_det_dogs)[0]:
    print(dog_files_short[idx])
    show_face(dog_files_short[idx])

We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Test performance of anotherface detection algorithm.
### Feel free to use as many code cells as needed.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F

# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    ## TODO: Complete the function.
    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image
    
    img_pil = Image.open(img_path)
    img_tensor = preprocess(img_pil)
    img_tensor.unsqueeze_(0)
    fc_out = VGG16(img_tensor.cuda())   
    output = F.softmax(fc_out)    
    values, indices = output.max(1)
    
    if use_cuda:
        return indices[0].cpu().numpy()#res # predicted class index
    else:
        return indices[0].numpy()

In [ ]:
print(VGG16_predict(dog_files[0]))

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    ## TODO: Complete the function.
    class_id = VGG16_predict(img_path)    
    return class_id in range(151, 269) # true/false

### (IMPLEMENTATION) Assess the Dog Detector

__Question 2:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 


In [ ]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_det_res = [dog_detector(dog_files_short[i]) for i in tqdm(range(len(dog_files_short)))]
print('Dog detector on dogs:', len(np.nonzero(dog_det_res)[0]), '%')

dog_det_res = [dog_detector(human_files_short[i]) for i in tqdm(range(len(human_files_short)))]
print('Dog detector on humans:', len(np.nonzero(dog_det_res)[0]), '%')

We suggest VGG-16 as a potential network to detect dog images in your algorithm, but you are free to explore other pre-trained networks (such as [Inception-v3](http://pytorch.org/docs/master/torchvision/models.html#inception-v3), [ResNet-50](http://pytorch.org/docs/master/torchvision/models.html#id3), etc).  Please use the code cell below to test other pre-trained PyTorch models.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Report the performance of another pre-trained network.
### Feel free to use as many code cells as needed.

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 10%.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dog_images/train`, `dog_images/valid`, and `dog_images/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [2]:
import os
import torch
from torchvision import datasets
import numpy as np
import torchvision.transforms as transforms

# check if CUDA is available
use_cuda = torch.cuda.is_available()

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

train_data = datasets.ImageFolder("data/dogImages/train/", transform=preprocess)
valid_data = datasets.ImageFolder("data/dogImages/valid/", transform=preprocess)
test_data = datasets.ImageFolder("data/dogImages/test/", transform=preprocess)

# define dataloader parameters
batch_size = 32
num_workers=0

# prepare data loaders
loaders_scratch = {}
loaders_scratch['train'] = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
loaders_scratch['valid'] = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
loaders_scratch['test'] = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [3]:
torch.cuda.is_available()

True

In [4]:
# load imagenet 1000 classes dict
import json
classes_path = "classes_imagenet.json"
with open(classes_path) as json_file:
    classes_dict = json.load(json_file)
classes_dict = {int(k): v for k, v in classes_dict.items()}
# print(classes_dict)
classes_dogs = {k : v for k, v in classes_dict.items() if k in list(range(151, 270))}
print(classes_dogs)

{151: 'Chihuahua', 152: 'Japanese spaniel', 153: 'Maltese dog, Maltese terrier, Maltese', 154: 'Pekinese, Pekingese, Peke', 155: 'Shih-Tzu', 156: 'Blenheim spaniel', 157: 'papillon', 158: 'toy terrier', 159: 'Rhodesian ridgeback', 160: 'Afghan hound, Afghan', 161: 'basset, basset hound', 162: 'beagle', 163: 'bloodhound, sleuthhound', 164: 'bluetick', 165: 'black-and-tan coonhound', 166: 'Walker hound, Walker foxhound', 167: 'English foxhound', 168: 'redbone', 169: 'borzoi, Russian wolfhound', 170: 'Irish wolfhound', 171: 'Italian greyhound', 172: 'whippet', 173: 'Ibizan hound, Ibizan Podenco', 174: 'Norwegian elkhound, elkhound', 175: 'otterhound, otter hound', 176: 'Saluki, gazelle hound', 177: 'Scottish deerhound, deerhound', 178: 'Weimaraner', 179: 'Staffordshire bullterrier, Staffordshire bull terrier', 180: 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier', 181: 'Bedlington terrier', 182: 'Border terrier', 183: 'Kerry blue terrier

In [14]:
import torch.nn as nn
import torch.nn.functional as F
from torch import sigmoid

# NCLASSES = 269 - 151
NCLASSES = 133
DEBUG = 0

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    # A simple AlexNet architecture as baseline model
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 48, 11, 4, padding=2)
        self.conv2 = nn.Conv2d(48, 128, 5, 1, padding=2)
        self.conv3 = nn.Conv2d(128, 192, 3, 1, padding=1)
        self.conv4 = nn.Conv2d(192, 192, 3, 1, padding=1)
        self.conv5 = nn.Conv2d(192, 128, 3, 1, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(6 * 6 * 128, 2048, bias=True)
        self.fc2 = nn.Linear(2048, 2048, bias=True)
        self.fc3 = nn.Linear(2048, NCLASSES, bias=True)
        
        self.dropout = nn.Dropout(0.5)
        
    
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = x.view(-1, 6 * 6 * 128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = sigmoid(x)
        
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()
print(model_scratch)

Net(
  (conv1): Conv2d(3, 48, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (conv2): Conv2d(48, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=133, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [15]:
print("Total number of memory allocated on GPU: {:.2f} MB".format(torch.cuda.max_memory_allocated() * 1.0e-6))

Total number of memory allocated on GPU: 400.39 MB


__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__ 

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [20]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01)

In [17]:
from tensorboardX import SummaryWriter
from pathlib import Path

lr = optimizer_scratch.param_groups[0]['lr']
log_path = 'tensorboard_logs'
checkpoints_dir = 'checkpoints'
comment = "scratch-"
checkpoint_name = '{}model-batch-{}-lr-{}'.format(
                comment, batch_size, lr)
current_checkpoint_dir = Path(checkpoints_dir) / checkpoint_name
checkpoints_path = str(current_checkpoint_dir / '{}.pt'.format(checkpoint_name))
Path(checkpoints_path).parent.mkdir(parents=True, exist_ok=True)  # create checkpoint folder
tensorboard_dir = str(Path(log_path) / checkpoint_name)
writer = SummaryWriter(tensorboard_dir)  # creating TensorBoard writer

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [18]:
def save_checkpoint(checkpoint: dict, save_path: str, checkpoint_type: str = 'best'):
    checkpoint_type = checkpoint_type.replace(' ', '_')
    if len(checkpoint_type) != 0:
        if checkpoint_type[-1] != '-':
            checkpoint_type += '-'  # 'best' -> 'best-' OR 'epoch_10' -> 'epoch_10-'

    save_path = str(Path(save_path).parent / '{}{}'.format(checkpoint_type, Path(save_path).name))

    torch.save(checkpoint, save_path)  # save checkpoint
    print("SUCCESSFULLY SAVED '{}' checkpoint to: {}".format(checkpoint_type, save_path))

In [19]:
def load_checkpoint(checkpoint_path, model=None, optimizer=None):
    
    """
    :param checkpoint_path: path to save checkpoint
    :param model: model that we want to load checkpoint parameters into
    :param optimizer: optimizer we defined in previous training
    :return:
    """
    assert Path(checkpoint_path).exists(), "NOT EXISTS the checkpoint_path: {}".format(checkpoint_path)
    checkpoint_dict = torch.load(checkpoint_path)  # load checkpoint file
    print("\nSUCCESSFULLY LOADED checkpoint from {}".format(checkpoint_path))
    print("Available checkpoint's dictionary keys: {}".format(checkpoint_dict.keys()))

    result_dict = dict()
    if 'datetime_saved' in checkpoint_dict.keys():
        result_dict['datetime_saved'] = checkpoint_dict['datetime_saved']
    else:
        result_dict['datetime_saved'] = None

    if 'epoch' in checkpoint_dict.keys():
        result_dict['epoch'] = checkpoint_dict['epoch']
    else:
        result_dict['epoch'] = None

    if 'loss_train' in checkpoint_dict.keys():
        result_dict['loss_train'] = checkpoint_dict['loss_train']
    else:
        result_dict['loss_train'] = None

    if 'loss_val' in checkpoint_dict.keys():
        result_dict['loss_val'] = checkpoint_dict['loss_val']
    else:
        result_dict['loss_val'] = None
        
    if 'lr' in checkpoint_dict.keys():
        result_dict['lr'] = checkpoint_dict['lr']
    else:
        result_dict['lr'] = None

    if 'num_classes' in checkpoint_dict.keys():
        result_dict['num_classes'] = checkpoint_dict['num_classes']
    else:
        result_dict['num_classes'] = None

    if 'model_state_dict' in checkpoint_dict.keys():
        _model_state_dict = checkpoint_dict['model_state_dict']
    else:
        _model_state_dict = None

    if 'optimizer_state_dict' in checkpoint_dict.keys():
        _optimizer_state_dict = checkpoint_dict['optimizer_state_dict']
    else:
        _optimizer_state_dict = None

    if 'end_training_time' in checkpoint_dict.keys():
        result_dict['end_training_time'] = checkpoint_dict['end_training_time']
    else:
        result_dict['end_training_time'] = None

    if 'total_train_time_seconds' in checkpoint_dict.keys():
        result_dict['total_train_time_seconds'] = checkpoint_dict['total_train_time_seconds']
    else:
        result_dict['total_train_time_seconds'] = None
        
    if 'classes_dict' in checkpoint_dict.keys():
        result_dict['classes_dict'] = checkpoint_dict['classes_dict']
    else:
        result_dict['classes_dict'] = None

    if 'tensorboard_dir' in checkpoint_dict.keys():
        result_dict['tensorboard_dir'] = checkpoint_dict['tensorboard_dir']
    else:
        result_dict['tensorboard_dir'] = None

    if model is None:  # if model not defined, than model should be initialized to load in the checkpoint parameters
        assert _model_state_dict is not None, 'NOT FOUNDED model_state_dict in checkpoint dict'
        
        model = Net()
        model.load_state_dict(_model_state_dict)  # loading parameters from saved state_dict to initialized model
    else:
        assert _model_state_dict is not None, 'NOT FOUNDED model_state_dict in checkpoint dict'
        model.load_state_dict(_model_state_dict)  # loading parameters from saved state_dict to defined model
        
    if optimizer is None:
        assert _optimizer_state_dict is not None, 'NOT FOUNDED optimizer_state_dict in checkoint dict'
        optimizer = optim.SGD(model.parameters(), lr=lr)
        optimizer.load_state_dict(_optimizer_state_dict)
    else:
        assert _optimizer_state_dict is not None, 'NOT FOUNDED optimizer_state_dict in checkoint dict'
        optimizer.load_state_dict(_optimizer_state_dict)

    return model, optimizer, result_dict

In [23]:
# if load model
load_model = True
load_last = False
if load_model:
    if load_last:
        # loading last checkpoint
        checkpoint_path = Path(checkpoints_path).parent / 'last-{}.pt'.format(checkpoint_name)
    else:
        # loading best checkpoint
        checkpoint_path = Path(checkpoints_path).parent / 'best-{}.pt'.format(checkpoint_name)
    model_scratch, optimizer_scratch, result_dict = load_checkpoint(checkpoint_path)
    if use_cuda:
        model_scratch.cuda()
    print("Loaded model. Epoch: {}, val_batch_loss: {}".format(result_dict['epoch'], result_dict['loss_val']))


SUCCESSFULLY LOADED checkpoint from checkpoints/scratch-model-batch-32-lr-0.1/best-scratch-model-batch-32-lr-0.1.pt
Available checkpoint's dictionary keys: dict_keys(['datetime_saved', 'epoch', 'loss_train', 'loss_val', 'lr', 'num_classes', 'model_state_dict', 'optimizer_state_dict', 'end_training_time', 'total_train_time_seconds', 'classes_dict', 'tensorboard_dir'])
Loaded model. Epoch: 94, val_batch_loss: 4.673764726191616


In [21]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda,
         resume_train=False, result_dict=None):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    
    if resume_train:
        assert result_dict is not None
        
    valid_loss_min = np.Inf 
    total_train_time_seconds = 0.0
    epoch_begin = 1
    
    if resume_train:
        print("Resuming training process. Last epoch: {}, valid loss min: {}".
             format(result_dict['epoch'], result_dict['loss_val']))
        valid_loss_min = result_dict['loss_val']
        total_train_time_seconds = result_dict['total_train_time_seconds']
        epoch_begin = result_dict['epoch'] + 1
    else:
        print("Starting train from epoch 1, from scratch")
        
    
    start_training_time = datetime.now()
    
    for epoch in range(epoch_begin, epoch_begin + n_epochs):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        if writer is not None:
            writer.add_scalar('Learning Rate', lr, epoch)
        
        
        ###################
        # train the model #
        ###################
        model.train()
        with tqdm(loaders['train'], unit="batch") as tepoch:
            for data, target in tepoch:
                tepoch.set_description(f"Train. Epoch {epoch}")
    #             print(target)
    #             print("data shape: {}".format(data.shape))
    #             print("target shape: {}".format(target.shape))
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## find the loss and update the model parameters accordingly
                ## record the average training loss, using something like
                ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
                
                optimizer_scratch.zero_grad()
                output = model_scratch(data)
                loss = criterion_scratch(output, target)
                loss.backward()
                optimizer_scratch.step()
                train_loss += loss.item() * data.size(0)
                tepoch.set_postfix(loss=train_loss)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        with tqdm(loaders['valid'], unit="batch") as vepoch:
            for data, target in vepoch:
                vepoch.set_description(f"Valid. Epoch {epoch}")
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                
                output = model_scratch(data)
                loss = criterion_scratch(output, target)
                ## update the average validation loss
                valid_loss += loss.item()*data.size(0)
                vepoch.set_postfix(loss=valid_loss)

        ## calculate average batch train and validation loss
        train_loss /= len(loaders['train'].sampler)
        valid_loss /= len(loaders['valid'].sampler)
    
        if writer is not None:
            writer.add_scalar('{}/TrainLoss'.format("train"), train_loss, epoch)
            writer.add_scalar('{}/ValidLoss'.format("val"), valid_loss, epoch)
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print("Epoch {}: Upgrate of model quality: ({:.5f})->({:.5f})".format(epoch, valid_loss_min, valid_loss))
            
            delta_time = datetime.now() - start_training_time
            checkpoint = {
                    'datetime_saved': datetime.now(),
                    'epoch': epoch,
                    'loss_train': train_loss, 
                    'loss_val': valid_loss,
                    'lr': lr, 
                    'num_classes': model.fc3.out_features,  
                    'model_state_dict': model_scratch.state_dict(), 
                    'optimizer_state_dict': optimizer_scratch.state_dict(), 
                    'end_training_time': delta_time, 
                    'total_train_time_seconds': total_train_time_seconds + 
                                                delta_time.seconds + 
                                                delta_time.days * 24 * 3600,
                    'classes_dict': classes_dict, 
                    'tensorboard_dir': tensorboard_dir
                }
            save_checkpoint(checkpoint=checkpoint, save_path=checkpoints_path, 
                           checkpoint_type='best')
#             torch.save(model_scratch.state_dict(), 'model_scratch.pt')
            valid_loss_min = valid_loss
    
        # save last checkpoint:
        delta_time = datetime.now() - start_training_time
        checkpoint = {
                'datetime_saved': datetime.now(),
                'epoch': epoch,
                'loss_train': train_loss, 
                'loss_val': valid_loss,
                'lr': lr, 
                'num_classes': model.fc3.out_features,  
                'model_state_dict': model_scratch.state_dict(), 
                'optimizer_state_dict': optimizer_scratch.state_dict(), 
                'end_training_time': delta_time, 
                'total_train_time_seconds': total_train_time_seconds + 
                                                delta_time.seconds + 
                                                delta_time.days * 24 * 3600,
                'classes_dict': classes_dict, 
                'tensorboard_dir': tensorboard_dir
        }
        save_checkpoint(checkpoint=checkpoint, save_path=checkpoints_path, 
                           checkpoint_type='last')
    
            
    # return trained model
    return model

In [22]:
# train the model
num_epochs = 300
# model_scratch = train(num_epochs, loaders_scratch, model_scratch, optimizer_scratch, 
#                       criterion_scratch, use_cuda, resume_train=True, result_dict=result_dict)

model_scratch = train(num_epochs, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, resume_train=False, result_dict=None)

# load the model that got the best validation accuracy
# model_scratch.load_state_dict(torch.load('model_scratch.pt'))

  0%|          | 0/209 [00:00<?, ?batch/s]

Starting train from epoch 1, from scratch


Valid. Epoch 1: 100%|██████████| 27/27 [00:07<00:00,  3.71batch/s, loss=4.09e+3]


Epoch: 1 	Training Loss: 4.890361 	Validation Loss: 4.890307
Epoch 1: Upgrate of model quality: (inf)->(4.89031)
SUCCESSFULLY SAVED 'best-' checkpoint to: checkpoints/scratch-model-batch-32-lr-0.1/best-scratch-model-batch-32-lr-0.1.pt


  0%|          | 0/209 [00:00<?, ?batch/s]

...

Epoch: 299 	Training Loss: 4.069098 	Validation Loss: 4.736155


  0%|          | 0/209 [00:00<?, ?batch/s]

SUCCESSFULLY SAVED 'last-' checkpoint to: checkpoints/scratch-model-batch-32-lr-0.1/last-scratch-model-batch-32-lr-0.1.pt


Valid. Epoch 300: 100%|██████████| 27/27 [00:07<00:00,  3.69batch/s, loss=3.96e+3]


Epoch: 300 	Training Loss: 4.064142 	Validation Loss: 4.737166
SUCCESSFULLY SAVED 'last-' checkpoint to: checkpoints/scratch-model-batch-32-lr-0.1/last-scratch-model-batch-32-lr-0.1.pt


### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 10%.

In [24]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    
    with tqdm(loaders['test'], unit="batch") as ttepoch:
        for data, target in ttepoch:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss += loss.item() * data.size(0)
    #         test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            ttepoch.set_postfix(loss=test_loss, accuracy=f"{correct*100.0/total:.2f}%")
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [25]:
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

100%|██████████| 27/27 [00:07<00:00,  3.59batch/s, accuracy=3.59%, loss=3.91e+3]

Test Loss: 3907.267307


Test Accuracy:  3% (30/836)


---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [ ]:
## TODO: Specify data loaders


### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 


if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:
criterion_transfer = None
optimizer_transfer = None

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
model_transfer = # train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
#model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    return None

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `human_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    


---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)